In [54]:
import numpy as np

In [55]:
# mass flow rate of cold side
m_c = 0.5

# mass flow rate of hot side
m_h = 0.48

# number of tubes
N = 13

# tube pitch
Y = 14*10**(-3)

# number of baffles
N_b = 9

# triangular(tri) / square(sq) pitch
what_pitch = 'sq'

In [56]:
# geometries of the tube
d_i = 6*10**(-3)
d_o = 8*10**(-3)

# geometries of the shell
L = 0.35
D_sh = 64*10**(-3)

# properties of water at 40 degrees
mu = 0.651*10**(-3)
k = 0.632
rho = 1/0.001008
cp = 4179
Pr = mu*cp/k

# thermal conductivity of tube
k_tube = 386

In [57]:
# mass flow rate in one tube
m_tube = m_h/N

# tube flow velocity
v_tube = m_tube/(rho*np.pi/4*d_i**2)

# Reynolds number in tube
Re_t = v_tube*rho*d_i/mu

# inner Nusselt number
Nu_i = 0.023*Re_t**0.8*Pr**0.3

# inner heat transfer coefficient
h_i = Nu_i*k/d_i

In [58]:
# baffle spacing
B = L/(N_b+1)

# area of shell
A_sh = D_sh*(Y-d_o)*B/Y

# flow velocity in shell
v_sh = m_c/(rho*A_sh)

# Reynolds number in shell
Re_sh = v_sh*d_o*rho/mu

# outer Nusselt number
if what_pitch == 'tri':
    Nu_o = 0.2*Re_sh**0.6*Pr**0.3
elif what_pitch == 'sq':
    Nu_o = 0.15*Re_sh**0.6*Pr**0.3

# outer heat transfer coefficient
h_o = Nu_o*k/d_o

In [89]:
# overall heat transfer coefficient
U = 1/ ( 1/h_i + d_i/(d_o*h_o) + d_i*np.log(d_o/d_i)/(2*k_tube) )

# total area of tube
A_i = np.pi*d_i*L*N

# start iteration
# log mean temperature diferrence (initial value)
T_lm = 0
T_lm_new = 40

while abs(T_lm - T_lm_new)>0.001:
    T_lm = T_lm_new
    T_c_out = 20+ T_lm*A_i*U/m_c/cp
    T_h_out = 60- T_lm*A_i*U/m_h/cp
    delta_T1 = 60- T_c_out
    delta_T2 = T_h_out - 20
    T_lm_new = (delta_T1-delta_T2)/np.log(delta_T1/delta_T2)

# overall heat transfer
Q = m_c*cp*(T_c_out-20)

# effectiveness
if m_c<m_h:
    eff = (T_c_out-20)/40
else:
    eff = (60-T_h_out)/40